**Biomedical Software Engineering**

**Prof. Arthur Goldberg**

**Dept. Genetics and Genomic Sciences**

**Spring 1, 2020**

# Advanced Python: using floats

## Rounding errors, and comparison
Floating-point numbers are represented in computer hardware as base 2 (binary) fractions.
Some numbers can be represented exactly.
(Source: [Python Tutorial, Chap. 15. Floating Point Arithmetic: Issues and Limitations](https://docs.python.org/3/tutorial/floatingpoint.html))

In [12]:
print(0.125)
print(f"{0.125:0.20f}")
assert 0.125 == 1/8
1/8 == 0/2 + 0/4 + 1/8

0.125
0.12500000000000000000


True

Virtually all modern chips use 64-bit IEEE standard 754-1985 floating format: a sign bit, an 11-bit exponent, and 52 bits of mantissa (about 16 decimal digits). Floats are converted into closest 64-bit representation.

1/10 is an infinitely repeating binary fraction, which cannot be represented exactly.


In [13]:
# the 16 digits of precision 0.1
# 0.1 is APPROXIMATELY represented
print(f"{0.1:0.17f}")
# all decimal digits in 0.1
print(f"{0.1:0.60f}")

0.10000000000000001
0.100000000000000005551115123125782702118158340454101562500000


Since different floats have different rounding errors in their representation, they may not compare as expected. Except in certain special cases, it's not a good idea to compare floats for equality. E.g., does 0.1 + 0.1 + 0.1 = 0.3?

In [14]:
0.1 + 0.1 + 0.1 == 0.3    # nope!

False

In [4]:
# math can help
import math
# sum of 10 copies of 0.1
print(sum([0.1] * 10))
# math.fsum(iterable)
# Return an accurate floating point sum of values in the iterable
math.fsum([0.1] * 10) == 1.0

0.9999999999999999


True

But that doesn't make it easy to compare floats in general.

### Use relative and absolute tolerances to compare floats
[math.isclose(a, b, *, rel_tol=1e-09, abs_tol=0.0)](https://docs.python.org/3/library/math.html#math.isclose)

Return `True` if the values a and b are close to each other and `False` otherwise.

Whether or not two values are considered close is determined according to given absolute and relative tolerances.

*rel_tol* is the relative tolerance – it is the maximum allowed difference between *a* and *b*, relative to the larger absolute value of *a* or *b*. *rel_tol* must be greater than zero.

*abs_tol* is the minimum absolute tolerance – useful for comparisons near zero. *abs_tol* must be at least zero.

If no errors occur, the result will be: `abs(a-b) <= max(rel_tol * max(abs(a), abs(b)), abs_tol)`.


See also [PEP 485 – A function for testing approximate equality](https://www.python.org/dev/peps/pep-0485/).

In [5]:
# compare floats
from math import isclose
print((0.1 + 0.1 + 0.1) - 0.3)  # the difference between two ways of computing 0.3
isclose(0.1 + 0.1 + 0.1, 0.3)

5.551115123125783e-17


True

In [15]:
# let's compare some pairs of numbers
pairs = (
    (1.000001E10, 1.000000001E10),    # large numbers
    (1.000001E-10, 1.000000001E-10),  # small numbers
    (0.0, -0.0),                      # yes, we have plus and minus zero
    (float('inf'), float('inf')),
    (-float('inf'), -float('inf')),
    (1.0, float('NaN')),
    (float('NaN'), float('NaN')),     # nothing is equal to (or close to) NaN (not a number)
)
# run isclose() on these pairs
for pair in pairs:
    first, second = pair
    print(f"first={first:.15e} second={second:.15e} {isclose(first, second)}")

first=1.000001000000000e+10 second=1.000000001000000e+10 False
first=1.000001000000000e-10 second=1.000000001000000e-10 False
first=0.000000000000000e+00 second=-0.000000000000000e+00 True
first=inf second=inf True
first=-inf second=-inf True
first=1.000000000000000e+00 second=nan False
first=nan second=nan False


Suppose we want our pairs of big and small numbers to be evaluated as *close*. What values of *rtol* and *atol* should we use?
First, the big numbers.

In [16]:
# investigate the big & small numbers
big_1, big_2 = 1.000001E10, 1.000000001E10

# functions to compare pairs of numbers, like isclose() does
# isclose() returns: abs(a-b) <= max(rel_tol * max(abs(a), abs(b)), abs_tol) 
def abs_diff(x, y):
    return abs(x - y)
print(f"abs_diff {abs_diff(big_1, big_2)}")

def rel_diff(x, y):
    larger = max(abs(x), abs(y))
    return abs(x - y)/larger

print(f"rel_diff {rel_diff(big_1, big_2)}")
# make rel_tol & abs_tol large enough to so that isclose(big_1, big_2) holds
print(isclose(big_1, big_2, rel_tol=1e-5, abs_tol=1e5))
# make if either rel_tol OR abs_tol is big enough, then isclose(big_1, big_2) will holds
print(isclose(big_1, big_2, rel_tol=1e-5))
isclose(big_1, big_2, abs_tol=1e5)

abs_diff 9990.0
rel_diff 9.98999001000999e-07
True
True


True

Let's look at the small numbers.

In [8]:
small_1, small_2 = 1.000001E-10, 1.000000001E-10
print(f"abs_diff {abs_diff(small_1, small_2)}")
print(f"rel_diff {rel_diff(small_1, small_2)}")
print(isclose(small_1, small_2, rel_tol=1e-6, abs_tol=1e-15))
# in this case neither rel_tol nor abs_tol are big enough
isclose(small_1, small_2, rel_tol=1e-8, abs_tol=1e-18)

abs_diff 9.989999999627648e-17
rel_diff 9.989990009637639e-07
True


False

## Exact calculations with Decimal
(Although the documentation meanders terribly, see the [Decimal Quick Start Tutorial](https://docs.python.org/3.8/library/decimal.html#quick-start-tutorial))

In [9]:
from decimal import *
print(3*0.1 - 0.3)
# the argment to Decimal is a string so that the number can be exact
decimal_0p1 = Decimal('0.1')
print(f"{decimal_0p1:.40f}")
decimal_0p3 = Decimal('0.3')
print(f"{decimal_0p3:.40f}")
# In Decimal, 3 * 0.1 == 0.3
3 * decimal_0p1 == decimal_0p3

5.551115123125783e-17
0.1000000000000000000000000000000000000000
0.3000000000000000000000000000000000000000


True

Suppose you want to make an infinite sequence of evenly spaced floating point values, e.g., 0, 0.1, 0.2, ... . `NumPy` [`linspace`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linspace.html) doesn't evenly space floating values, or make an infinite sequence.
I wanted an infinite sequence of evenly spaced floating point values so that my simulator could take checkpoints at exact, evenly spaced times.

In [10]:
import numpy as np
# numpy linspace isn't exact
[f"{v:.20f}, " for v in np.linspace(0, 1, 11)]

['0.00000000000000000000, ',
 '0.10000000000000000555, ',
 '0.20000000000000001110, ',
 '0.30000000000000004441, ',
 '0.40000000000000002220, ',
 '0.50000000000000000000, ',
 '0.60000000000000008882, ',
 '0.70000000000000006661, ',
 '0.80000000000000004441, ',
 '0.90000000000000002220, ',
 '1.00000000000000000000, ']

Using `Decimal`, solved this problem with `UniformSequence`, which inherits from `Iterator`.

In [11]:
""" Generate an infinite sequence of evenly spaced values

:Author: Arthur Goldberg <Arthur.Goldberg@mssm.edu>
:Date: 2019-12-11
:Copyright: 2019, Karr Lab
:License: MIT
"""

from decimal import Decimal, getcontext
import collections

UNIFORM_SEQ_PRECISION = 6


class UniformSequence(collections.abc.Iterator):
    """ Generate an infinite sequence of evenly spaced values, especially for non-integral step sizes

    The start and step size must be an integer, a float or string that can be represented as a Decimal
    with a mantissa that contains no more than `UNIFORM_SEQ_PRECISION` digits.
    Avoids floating-point roundoff errors by using a :obj:`Decimal` to represent the step size.

    Attributes:
        _start (:obj:`Decimal`): starting point of the sequence
        _step (:obj:`Decimal`): step size for the sequence
        _num_steps (:obj:`int`): number of steps taken in the sequence
    """

    def __init__(self, start, step):
        """ Initialize a :obj:`UniformSequence`

        Args:
            start (:obj:`str` or :obj:`number`): starting point of the sequence
            step (:obj:`str` or :obj:`number`): step size for the sequence

        Raises:
            :obj:`ValueError`: if the step size is 0, NaN, or infinite, or
                if the precision in `start` or `step` exceeds `UNIFORM_SEQ_PRECISION`
        """
        getcontext().prec = UNIFORM_SEQ_PRECISION
        self._start = Decimal(start)
        self._step = Decimal(step)
        if not self._step.is_normal():
            raise ValueError(f"UniformSequence: step={step} can't be 0, NaN, infinite, or subnormal")

        # start and step cannot contain more digits than the Decimal precision
        if UNIFORM_SEQ_PRECISION < len(self._start.as_tuple().digits):
            raise ValueError(f"UniformSequence: precision in start={start} exceeds UNIFORM_SEQ_PRECISION "
                             f"threshold={UNIFORM_SEQ_PRECISION}")
        if UNIFORM_SEQ_PRECISION < len(self._step.as_tuple().digits):
            raise ValueError(f"UniformSequence: precision in step={step} exceeds UNIFORM_SEQ_PRECISION "
                             f"threshold={UNIFORM_SEQ_PRECISION}")
        self._num_steps = 0

    def __iter__(self):
        """ Get this :obj:`UniformSequence`

        Returns:
            :obj:`UniformSequence`: this :obj:`UniformSequence`
        """
        return self

    def __next__(self):
        """ Get next value in the sequence

        Returns:
            :obj:`Decimal`: next value in this :obj:`UniformSequence`
        """
        # This expression is evaluated using exact Decimal math
        next_value = self._start + self._num_steps * self._step # this is the key line
        self._num_steps += 1
        if next_value.is_zero():
            return 0.
        return next_value

us = UniformSequence(0, '0.1')
for _ in range(10):
  print(f"{us.__next__():.20f}")

0.00000000000000000000
0.10000000000000000000
0.20000000000000000000
0.30000000000000000000
0.40000000000000000000
0.50000000000000000000
0.60000000000000000000
0.70000000000000000000
0.80000000000000000000
0.90000000000000000000


In [21]:
# UniformSequence raises an exception if the start or step cannot be represented exactly
# UniformSequence(0, 0.1)
import math
UniformSequence(math.sqrt(2), 1)
us = UniformSequence(0, '0.1')

ValueError: ignored